In [159]:
from pymongo import MongoClient
import pandas as pd
import re
import requests
import json


client = MongoClient('mongodb://localhost:27017/')
db = client.company


In [160]:
#query to filter database by deadpooled year, employees, founded year, lat and long. 

query= db.company.find({
    "$and":
    [{"deadpooled_year":None},
     {"number_of_employees":{"$ne":None}},
     {"founded_year":{"$gte":2007}},
     {"offices.longitude":{"$exists":True}},
     {"offices.latitude":{"$ne":None}},
     {"offices.longitude":{"$ne":None}},
     {"offices.longitude":{"$exists":True}
     }
    ]
}
)


In [162]:
print(query)

In [163]:
with_coords = pd.DataFrame(query)
with_coords.shape


(2152, 42)

In [164]:
#New DF with filtered columns
with_coords2 = with_coords[['_id','name','number_of_employees','category_code','offices','founded_year','ipo','funding_rounds','total_money_raised','investments','deadpooled_year','partners']]
with_coords2.head(100)
with_coords2.shape


(2152, 12)

In [178]:
#Get Offices

get_offices= pd.DataFrame(with_coords2['offices'].tolist()).stack().reset_index(level=1, drop=True)
office = get_offices.rename('office')
office.shape



(2292,)

In [189]:
#merge dataframe with new columns

df_clean= with_coords2.merge(office, left_index= True, right_index=True)

df_clean = df_clean[['name','number_of_employees','category_code','office','ipo','total_money_raised','founded_year','deadpooled_year','offices']]
df_clean.head(100)
print(df_clean[['office']])

                                                 office
0     {'description': 'HQ', 'address1': '539 Bryant ...
1     {'description': None, 'address1': '', 'address...
2     {'description': '', 'address1': '3525 Eastham ...
3     {'description': 'Livestream HQ', 'address1': '...
4     {'description': '', 'address1': '131 Varick St...
5     {'description': '', 'address1': '1459 18th Str...
6     {'description': None, 'address1': None, 'addre...
7     {'description': '', 'address1': '16161 Ventura...
8     {'description': 'Arts & Crafts', 'address1': '...
9     {'description': None, 'address1': None, 'addre...
10    {'description': '', 'address1': '53 S Knoll Rd...
11    {'description': '', 'address1': '1550 Bryant S...
12    {'description': None, 'address1': None, 'addre...
13    {'description': 'GoingOn Networks, Inc.', 'add...
14    {'description': '', 'address1': '', 'address2'...
15    {'description': '', 'address1': '', 'address2'...
16    {'description': 'boo-land', 'address1': 'A

In [190]:
#get latitude, longitude and office ID

def getOffices(offices):
    offices = offices['office']
    principal = None
    if offices['latitude'] and offices ['longitude']:
        
        principal = {
            "type":"Point",
            "coordinates":[offices['longitude'], offices['latitude']]
        }

        return {
            "long": offices['longitude'],
            "lat": offices['latitude'],
            "oficina": principal
        }
            

offices = df_clean[['office']].apply(getOffices, result_type="expand", axis=1)
offices.head(100)

,lat,long,oficina
0,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."
1,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
2,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
3,40.726155,-73.995625,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,40.801358,-74.337200,"{'type': 'Point', 'coordinates': [-74.3372, 40..."
5,37.762541,-122.397224,"{'type': 'Point', 'coordinates': [-122.397224,..."
6,37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,..."
7,37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,..."
8,52.374523,4.894862,"{'type': 'Point', 'coordinates': [4.8948623, 5..."
9,37.369650,-122.024210,"{'type': 'Point', 'coordinates': [-122.02421, ..."


In [191]:
#Concatenar dos dataframes

df_concat = pd.concat([df_clean,offices], axis=1)
df_concat.head()

,name,number_of_employees,category_code,office,ipo,total_money_raised,founded_year,deadpooled_year,offices,lat,long,oficina
0,Scribd,50,news,"{'description': 'HQ', 'address1': '539 Bryant ...",None,$25.8M,2007,None,"[{'description': 'HQ', 'address1': '539 Bryant...",37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."
1,MeetMoi,15,social,"{'description': None, 'address1': '', 'address...",None,$5.58M,2007,None,"[{'description': None, 'address1': '', 'addres...",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
2,Mahalo,40,web,"{'description': '', 'address1': '3525 Eastham ...",None,$21M,2007,None,"[{'description': '', 'address1': '3525 Eastham...",34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
3,Livestream,120,games_video,"{'description': 'Livestream HQ', 'address1': '...",None,$14.7M,2007,None,"[{'description': 'Livestream HQ', 'address1': ...",40.726155,-73.995625,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,AdaptiveBlue,15,games_video,"{'description': '', 'address1': '131 Varick St...",None,$24M,2007,None,"[{'description': '', 'address1': '131 Varick S...",40.801358,-74.337200,"{'type': 'Point', 'coordinates': [-74.3372, 40..."


In [192]:
#Get API for exchange rate

url = "https://api.exchangeratesapi.io/latest?base=USD"
res = requests.get(url)
rates = res.json()
print(rates)


{'rates': {'CAD': 1.3113190731, 'HKD': 7.8156862745, 'ISK': 126.4705882353, 'PHP': 51.4759358289, 'DKK': 6.6521390374, 'HUF': 290.4634581105, 'CZK': 22.7852049911, 'GBP': 0.8014973262, 'RON': 4.2188057041, 'SEK': 9.4557040998, 'IDR': 14130.0, 'INR': 68.5837789661, 'BRL': 3.7954545455, 'RUB': 63.5200534759, 'HRK': 6.5895721925, 'JPY': 108.9126559715, 'THB': 30.825311943, 'CHF': 0.9922459893, 'EUR': 0.8912655971, 'MYR': 4.1385026738, 'BGN': 1.7431372549, 'TRY': 5.7395721925, 'CNY': 6.8829768271, 'NOK': 8.6390374332, 'NZD': 1.5148841355, 'ZAR': 14.1599821747, 'USD': 1.0, 'MXN': 19.1997326203, 'SGD': 1.3609625668, 'AUD': 1.444741533, 'ILS': 3.5721925134, 'KRW': 1181.1140819964, 'PLN': 3.8090909091}, 'base': 'USD', 'date': '2019-07-10'}


In [193]:
#get currency types


x = []
for i in df_concat['total_money_raised']:
    y=re.findall("^\D+",i)
    if y not in x:
        x.append(y)
        
print(x)

[['$'], ['€'], ['£'], ['C$']]


In [194]:
def dropSpaces(currency):
    return currency.rstrip()
    

with_coords2['total_money_raised'] = with_coords2['total_money_raised'].apply(dropSpaces)




/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [195]:
def coinsValues_USD(currency):
    curr_float = float(re.search("\d*\.\d*|\d+", currency)[0])    
    if currency.endswith("M"):
        curr_float= curr_float*1000000
    if currency.endswith("B"):
        curr_float= curr_float*1000000000
    if currency.endswith("k"):
        curr_float= curr_float*1000
    if currency.startswith("€"): 
        curr_float = curr_float*rates['rates']['EUR']
    if currency.startswith("$"): 
        curr_float = curr_float
    if currency.startswith("£"):
        curr_float = curr_float*rates['rates']['GBP']
    if currency.startswith("C$"):
        curr_float = curr_float*rates['rates']['SEK']
    return int(curr_float)

df_concat['total_USD_raised'] = df_concat['total_money_raised'].apply(coinsValues_USD)


df_concat.head(100)

,name,number_of_employees,category_code,office,ipo,total_money_raised,founded_year,deadpooled_year,offices,lat,long,oficina,total_USD_raised
0,Scribd,50,news,"{'description': 'HQ', 'address1': '539 Bryant ...",None,$25.8M,2007,None,"[{'description': 'HQ', 'address1': '539 Bryant...",37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",25800000
1,MeetMoi,15,social,"{'description': None, 'address1': '', 'address...",None,$5.58M,2007,None,"[{'description': None, 'address1': '', 'addres...",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",5580000
2,Mahalo,40,web,"{'description': '', 'address1': '3525 Eastham ...",None,$21M,2007,None,"[{'description': '', 'address1': '3525 Eastham...",34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,...",21000000
3,Livestream,120,games_video,"{'description': 'Livestream HQ', 'address1': '...",None,$14.7M,2007,None,"[{'description': 'Livestream HQ', 'address1': ...",40.726155,-73.995625,"{'type': 'Point', 'coordinates': [-73.995625, ...",14700000
4,AdaptiveBlue,15,games_video,"{'description': '', 'address1': '131 Varick St...",None,$24M,2007,None,"[{'description': '', 'address1': '131 Varick S...",40.801358,-74.337200,"{'type': 'Point', 'coordinates': [-74.3372, 40...",24000000
5,Pownce,6,web,"{'description': '', 'address1': '1459 18th Str...",None,$0,2007,None,"[{'description': '', 'address1': '1459 18th St...",37.762541,-122.397224,"{'type': 'Point', 'coordinates': [-122.397224,...",0
6,CriticalMetrics,4,web,"{'description': None, 'address1': None, 'addre...",None,$100k,2007,None,"[{'description': None, 'address1': None, 'addr...",37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,...",100000
7,SodaHead,25,web,"{'description': '', 'address1': '16161 Ventura...",None,$12.7M,2007,None,"[{'description': '', 'address1': '16161 Ventur...",37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,...",12700000
8,Wakoopa,8,web,"{'description': 'Arts & Crafts', 'address1': '...",None,$1M,2007,None,"[{'description': 'Arts & Crafts', 'address1': ...",52.374523,4.894862,"{'type': 'Point', 'coordinates': [4.8948623, 5...",1000000
9,coRank,0,web,"{'description': None, 'address1': None, 'addre...",None,$0,2007,None,"[{'description': None, 'address1': None, 'addr...",37.369650,-122.024210,"{'type': 'Point', 'coordinates': [-122.02421, ...",0


In [196]:
#reshape columns and get dimensions
df_concat = df_concat [['name','office','lat','long','oficina','category_code','number_of_employees','total_USD_raised','founded_year','deadpooled_year']]
df_concat.shape

      

(2292, 10)

In [198]:
#export json

df_concat.to_json('offices1.json', orient='records')


        